# Proses Gambar Sumber RAW Output A (modified)

In [28]:
import os
import csv
import cv2
import numpy as np
from tqdm import tqdm  # pastikan sudah install: pip install tqdm

import importlib
import lib.export
import lib.D_acc_modified
importlib.reload(lib.export)
importlib.reload(lib.D_acc_modified)
from lib.export import ekspor_png
from lib.D_acc_modified import get_ccf, get_ccf_wrapper, get_color_corrected_increased_dynamic_range

In [ ]:
pathfile = "../UIEB/raw-890/"
# pathfile = "../UIEB/results/2024_haofeng_hu_just_a/"
filename = "15001.png"
fpath = pathfile + filename

img = cv2.imread(fpath)
# metrics = get_ccf_wrapper(image_path, device) # metrics['CCF']/metrics['cast']
metrics = get_ccf(img, cie=True, device="cuda")
print(metrics)



im_increased, cast = get_color_corrected_increased_dynamic_range(fpath, device="cuda")
img_pil = (im_increased * 255).clip(0, 255).astype(np.uint8)
img_cv = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
metrics = get_ccf(img_cv, cie=True, device="cuda")
print("Before")
print(metrics)

ekspor_png(im_increased, "temp/","temp.png")
img_cv2 = cv2.imread("temp/temp.png")
im_increased, cast = get_color_corrected_increased_dynamic_range("temp/temp.png", device="cuda")
img_pil = (im_increased * 255).clip(0, 255).astype(np.uint8)
img_cv = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
metrics = get_ccf(img_cv2, cie=True, device="cuda")
print("After 1")
print(metrics)

ekspor_png(im_increased, "temp/","temp.png")
img_cv2 = cv2.imread("temp/temp.png")
im_increased, cast = get_color_corrected_increased_dynamic_range("temp/temp.png", device="cuda")
img_pil = (im_increased * 255).clip(0, 255).astype(np.uint8)
img_cv = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
metrics = get_ccf(img_cv2, cie=True, device="cuda")
print("After 2")
print(metrics)

ekspor_png(im_increased, "temp/","temp.png")
img_cv2 = cv2.imread("temp/temp.png")
im_increased, cast = get_color_corrected_increased_dynamic_range("temp/temp.png", device="cuda")
img_pil = (im_increased * 255).clip(0, 255).astype(np.uint8)
img_cv = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
metrics = get_ccf(img_cv2, cie=True, device="cuda")
print("After 3")
print(metrics)

{'CCF': 1.9417657810517601, 'cast': 'greenish'}
Before
{'CCF': 1.080490157805295, 'cast': 'yellowish'}
After 1
{'CCF': 1.068922354437344, 'cast': 'bluish'}
After 2
{'CCF': 0.9532736330962789, 'cast': 'no-color-cast'}
After 3
{'CCF': 0.9532736330962789, 'cast': 'no-color-cast'}


In [30]:
def batch_lab_metrics_resume(
    input_folder: str,
    process_name: str,
    cie: bool = True,
    device: str = "cuda",
    exts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"),
    recursive: bool = False,
    limit: int | None = None,
):
    """
    Batch processing dengan resume otomatis dari CSV + progress bar.
    - Baca CSV (jika ada) → skip file yang sudah diproses.
    - Proses hanya file sisa (bisa dibatasi 'limit').
    - Progress bar ditampilkan dengan tqdm.
    - Append hasil ke CSV (header ditulis kalau CSV belum ada).
    """
    # 1) Kumpulkan semua file gambar
    all_files = []
    walker = os.walk(input_folder) if recursive else [(input_folder, [], os.listdir(input_folder))]
    for root, _, files in walker:
        for fname in files:
            if fname.lower().endswith(exts):
                all_files.append(os.path.abspath(os.path.join(root, fname)))
    all_files.sort()

    done = set()

    # 3) Tentukan sisa file
    remaining = [p for p in all_files if p not in done]
    if limit is not None:
        remaining = remaining[:max(0, limit)]

    if not remaining:
        print(f"Tidak ada file baru untuk diproses. Total sudah diproses: {len(done)}.")
        return 0

    # 4) Proses file dengan progress bar
    for fpath in tqdm(remaining, desc="Processing images", unit="img"):
        filename = os.path.basename(fpath)

        im_increased, cast = get_color_corrected_increased_dynamic_range(fpath, device="cuda")
        ekspor_png(im_increased, "../UIEB/results/" + process_name, filename)


    print(f"Selesai. Diproses file baru. Total sebelumnya: {len(done)}. "
          f"Total sekarang (CSV) ≈ {len(done)}.")
    


In [31]:
# source_folder = "../UIEB/raw-890"
# out_folder = ""
# process_name = "2024_haofeng_hu_modified_a" #jadi folder dan nama file csv
# batch_lab_metrics_resume(source_folder, process_name, limit=890)